In [1]:

# Import common dependencies
import pandas as pd  # noqa
import numpy as np
import matplotlib  # noqa
import matplotlib.pyplot as plt
import datetime  # noqa
import PIL  # noqa
import glob  # noqa
import pickle  # noqa
from pathlib import Path  # noqa
from scipy import misc  # noqa
import sys
import tensorflow as tf
import pdb
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "2"
TRADE_COST_FRAC = .003
EPSILON = 1e-10
ADV_MULT = 1e-3

In [2]:
uni_tokens = set()
uni_commands = set()
uni_actions = set()
fname = 'tasks_with_length_tags.txt'
with open(fname) as f:
    content = f.readlines()
content2 = [c.split(' ') for c in content]
# you may also want to remove whitespace characters like `\n` at the end of each line
commands = []
actions = []
content = [l.replace('\n', '') for l in content]
commands = [x.split(':::')[1].split(' ')[1:-1] for x in content]
actions = [x.split(':::')[2].split(' ')[1:-2] for x in content]
structures = [x.split(':::')[3].split(' ')[2:] for x in content]

structures = [[int(l) for l in program] for program in structures]
#actions = [[wd.replace('\n', '') for wd in res] for res in actions]

In [3]:
max_actions_per_subprogram = max([max([s for s in struct]) for struct in structures]) + 1
max_num_subprograms = max([len(s) for s in structures]) + 1
max_cmd_len = max([len(s) for s in commands]) + 1
max_act_len = max([len(a) for a in actions]) + 1
cmd_lengths_list = [len(s)+1 for s in commands]
cmd_lengths_np = np.array(cmd_lengths_list)
max_num_subprograms, max_cmd_len, max_act_len, max_actions_per_subprogram


(7, 10, 49, 9)

In [4]:
def build_fmap_invmap(unique, num_unique):
    fmap = dict(zip(unique, range(num_unique)))
    invmap = dict(zip(range(num_unique), unique))
    return fmap, invmap


In [5]:
for li in content2:
    for wd in li:
        uni_tokens.add(wd)
for li in commands:
    for wd in li:
        uni_commands.add(wd)
for li in actions:
    for wd in li:
        uni_actions.add(wd)
uni_commands.add('end_command')
uni_actions.add('end_subprogram')
uni_actions.add('end_action')
num_cmd = len(uni_commands)
num_act = len(uni_actions)
command_map, command_invmap = build_fmap_invmap(uni_commands, num_cmd)
action_map, action_invmap = build_fmap_invmap(uni_actions, num_act)

In [6]:


def dense_scaled(prev_layer, layer_size, name=None, reuse=False, scale=1.0):
    output = tf.layers.dense(prev_layer, layer_size, reuse=reuse) * scale
    return output


def dense_relu(dense_input, layer_size, scale=1.0):
    dense = dense_scaled(dense_input, layer_size, scale=scale)
    output = tf.nn.leaky_relu(dense)

    return output

def get_grad_norm(opt_fcn, loss):
    gvs = opt_fcn.compute_gradients(loss)
    grad_norm = tf.sqrt(tf.reduce_sum(
        [tf.reduce_sum(tf.square(grad)) for grad, var in gvs if grad is not None]))
    return grad_norm


def apply_clipped_optimizer(opt_fcn, loss, clip_norm=.1, clip_single=.03, clip_global_norm=False):
    gvs = opt_fcn.compute_gradients(loss)

    if clip_global_norm:
        gs, vs = zip(*[(g, v) for g, v in gvs if g is not None])
        capped_gs, grad_norm_total = tf.clip_by_global_norm([g for g in gs], clip_norm)
        capped_gvs = list(zip(capped_gs, vs))
    else:
        grad_norm_total = tf.sqrt(
            tf.reduce_sum([tf.reduce_sum(tf.square(grad)) for grad, var in gvs if grad is not None]))
        capped_gvs = [(tf.clip_by_value(grad, -1 * clip_single, clip_single), var)
                      for grad, var in gvs if grad is not None]
        capped_gvs = [(tf.clip_by_norm(grad, clip_norm), var) for grad, var in capped_gvs if grad is not None]

    optimizer = opt_fcn.apply_gradients(capped_gvs)

    return optimizer, grad_norm_total


def mlp(x, hidden_sizes, output_size=None, name='', reuse=False):
    prev_layer = x

    for idx, l in enumerate(hidden_sizes):
        dense = dense_scaled(prev_layer, l, name='mlp' + name + '_' + str(idx))
        prev_layer = tf.nn.leaky_relu(dense)

    output = prev_layer

    if output_size is not None:
        output = dense_scaled(prev_layer, output_size, name='mlp' + name + 'final')

    return output

def mlp_with_adversaries(x, hidden_sizes, output_size=None, name='', reuse=False):
    prev_layer = x
    adv_phs = []
    for idx, l in enumerate(hidden_sizes):
        
        adversary = tf.placeholder_with_default(tf.zeros_like(prev_layer), prev_layer.shape)
        prev_layer = prev_layer + adversary
        adv_phs.append(adversary)
        
        dense = dense_scaled(prev_layer, l, name='mlp' + name + '_' + str(idx))
        prev_layer = tf.nn.leaky_relu(dense)

    output = prev_layer

    if output_size is not None:
        output = dense_scaled(prev_layer, output_size, name='mlp' + name + 'final')

    return output, adv_phs



In [7]:

commands_ind = [[command_map[c] for c in cmd] + [0] * (max_cmd_len - len(cmd)) for cmd in commands]
actions_ind = [[action_map[a] for a in act] + [0] * (max_act_len - len(act)) for act in actions]
cmd_np = np.array(commands_ind)
actions_structured = []
mask_structured = []
for row in range(len(structures)):
    mask_row = []
    action_row = []
    act = actions_ind[row]
    struct = structures[row]
    start = 0
    for step in struct:
        end = start + step
        a = act[start:end]
        padding = max_actions_per_subprogram - step - 1
        action_row.append(a + [action_map['end_action']] + [0] * padding)
        start = end
    actions_structured.append(
        action_row + [[action_map['end_subprogram']] + [0] * (max_actions_per_subprogram - 1)] +
        [[0] * max_actions_per_subprogram] * (max_num_subprograms - len(struct) - 1)
    )
act_np = np.array(actions_structured)
struct_padded = [[sa + 1 for sa in s] + [1] + [0] * (max_num_subprograms - len(s) - 1) for s in structures]
struct_np = np.array(struct_padded)

mask_list = [[np.concatenate((np.ones(st), np.zeros(max_actions_per_subprogram - st)), 0) 
              for st in s] for s in struct_np]
mask_np = np.array(mask_list)

In [8]:
tf.reset_default_graph()
default_sizes = 128
size_emb = 64
num_layers_encoder = 6
hidden_filters = 256
num_layers_subprogram = 3
hidden_filters_subprogram = 256
init_mag = 1e-3
cmd_mat = tf.Variable(init_mag*tf.random_normal([num_cmd, size_emb]))
act_mat = tf.Variable(init_mag*tf.random_normal([num_act, size_emb]))
act_st_emb = tf.Variable(init_mag*tf.random_normal([size_emb]))
global_bs = None
global_time_len = 7
action_lengths = None
max_num_actions= None
# global_bs = 8
global_time_len = 7
max_num_actions = 9
output_keep_prob = tf.placeholder_with_default(1.0, ())
state_keep_prob = tf.placeholder_with_default(1.0, ())
cmd_ind = tf.placeholder(tf.int32, shape=(global_bs, 10,))
act_ind = tf.placeholder(tf.int32, shape=(global_bs, global_time_len, 9))
mask_ph = tf.placeholder(tf.float32, shape=(global_bs, global_time_len, 9))
cmd_lengths = tf.placeholder(tf.int32, shape=(global_bs,))
act_lengths = tf.placeholder(tf.int32, shape=(global_bs, 7))
learning_rate = tf.placeholder(tf.float32, shape = (None))

cmd_emb = tf.nn.embedding_lookup(cmd_mat, cmd_ind)
act_emb = tf.nn.embedding_lookup(act_mat, act_ind)
tf_bs = tf.shape(act_ind)[0]
act_st_emb_expanded = tf.tile(tf.reshape(
    act_st_emb, [1, 1, 1, size_emb]), [tf_bs, global_time_len, 1, 1])
act_emb_with_st = tf.concat((act_st_emb_expanded, act_emb), 2)

first_cell_encoder = [tf.nn.rnn_cell.LSTMCell(
    hidden_filters, forget_bias=1., name = 'layer1_'+d) for d in ['f', 'b']]
hidden_cells_encoder = [[tf.nn.rnn_cell.LSTMCell(
    hidden_filters,forget_bias=1., name = 'layer' + str(lidx) + '_' + d)  for d in ['f', 'b']]
                        for lidx in range(num_layers_encoder - 1)]
hidden_cells_encoder = [[tf.nn.rnn_cell.DropoutWrapper(cell,
    output_keep_prob=output_keep_prob, state_keep_prob=state_keep_prob,
    variational_recurrent=True, dtype=tf.float32) for cell in cells] 
                        for cells in hidden_cells_encoder[:-1]] + [hidden_cells_encoder[-1]]
cells_encoder = [first_cell_encoder] + hidden_cells_encoder
c1, c2 = zip(*cells_encoder)
cells_encoder = [c1, c2]
def encode(x, num_layers, cells, initial_states, lengths, name='',):
    prev_layer = x
    shortcut = x
    hiddenlayers = []
    returncells = []
    cell_fw, cell_bw = cells
    bs = tf.shape(x)[0]
    for idx in range(num_layers):
        prev_layer, c = tf.nn.bidirectional_dynamic_rnn(
                cell_fw = cell_fw[idx],
                cell_bw = cell_bw[idx],
                inputs = prev_layer,
                sequence_length=lengths,
                initial_state_fw=None,
                initial_state_bw=None,
                dtype=tf.float32,
                scope='encoder'+str(idx)
            )
        prev_layer = tf.concat(prev_layer, 2)
        prev_layer = tf.nn.leaky_relu(prev_layer)
        returncells.append(c)
        hiddenlayers.append(prev_layer)
        if idx == num_layers - 1:
            #pdb.set_trace()
            output = tf.gather_nd(
                        prev_layer,
                        tf.stack([tf.range(bs), lengths], 1),
                        name=None
                    )
            return prev_layer, returncells, hiddenlayers, output
        prev_layer = tf.concat((prev_layer, shortcut), 2)
encoding_last_layer, encoding_final_cells, encoding_hidden_layers, encoding_last_timestep = encode(
    cmd_emb, num_layers_encoder, cells_encoder,None, lengths = cmd_lengths, name = 'encoder')
# encoding_last_timestep = encoding_last_layer[:,cmd_lengths, :]
hidden_filters_encoder = encoding_last_timestep.shape[-1].value
first_cell_subprogram = tf.nn.rnn_cell.LSTMCell(
    hidden_filters_subprogram, forget_bias=1., name = 'subpogramlayer1_')
hidden_cells_subprogram = [tf.nn.rnn_cell.LSTMCell(
    hidden_filters_subprogram,forget_bias=1., name = 'subpogramlayer' + str(lidx))
                        for lidx in range(num_layers_subprogram - 1)]

cells_subprogram_rnn = [first_cell_subprogram] + hidden_cells_subprogram

attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(
    num_units=hidden_filters_encoder, memory=encoding_last_layer,
    memory_sequence_length=cmd_lengths)
attention_mechanism = tf.contrib.seq2seq.LuongAttention(
    num_units=hidden_filters_encoder//2, memory=encoding_last_layer,
    memory_sequence_length=cmd_lengths)
cells_subprogram = [
    tf.contrib.seq2seq.AttentionWrapper(
        cell, attention_mechanism, attention_layer_size = hidden_filters_subprogram) 
    for cell in cells_subprogram_rnn]

def subprogram(x, num_layers, cells, initial_states, lengths, reuse, name='',):
    prev_layer = x
    shortcut = x
    hiddenlayers = []
    returncells = []
    bs = tf.shape(x)[0]
    for idx in range(num_layers):
        print(idx)
        if idx == 0:
            num_past_units = hidden_filters
        else:
            num_past_units = hidden_filters_subprogram
        with tf.variable_scope(name + 'subprogram' + str(idx), reuse=reuse):
#             self_attention_mechanism = tf.contrib.seq2seq.LuongAttention(
#                 num_units=num_past_units, memory=prev_layer,
#                 memory_sequence_length=tf.expand_dims(tf.range(10), 0))
#             cell_with_selfattention = tf.contrib.seq2seq.AttentionWrapper(
#                     cells[idx], self_attention_mechanism, attention_layer_size = num_past_units)

            prev_layer, c = tf.nn.dynamic_rnn(
                    cell = cells[idx],
                    inputs = prev_layer,
                    sequence_length=lengths,
                    initial_state = None,
                    dtype=tf.float32,
                )
            prev_layer = tf.concat(prev_layer, 2)
            prev_layer = tf.nn.leaky_relu(prev_layer)
            returncells.append(c)
            hiddenlayers.append(prev_layer)
            if idx == num_layers - 1:
                output = tf.gather_nd(
                            prev_layer,
                            tf.stack([tf.range(bs), lengths], 1),
                            name=None
                        )
                return prev_layer, returncells, hiddenlayers, output
            prev_layer = tf.concat((prev_layer, shortcut), 2)
encodings = [encoding_last_timestep]
last_encoding = encoding_last_timestep
initial_cmb_encoding = last_encoding
loss = 0
action_probabilities_presoftmax = []
for sub_idx in range(max_num_subprograms): 
    from_last_layer = tf.tile(tf.expand_dims(tf.concat((
        last_encoding, initial_cmb_encoding), 1), 1), [1, max_num_actions + 1, 1])
    autoregressive = act_emb_with_st[:,sub_idx, :, :]
    x_input = tf.concat((from_last_layer, autoregressive), -1)
    subprogram_last_layer, _, subprogram_hidden_layers, subprogram_output = subprogram(
        x_input, num_layers_subprogram, cells_subprogram,None, 
        lengths = act_lengths[:, sub_idx], reuse = (sub_idx > 0), name = 'subprogram')
    action_prob_flat = mlp(
        tf.reshape(subprogram_last_layer, [-1, hidden_filters_subprogram]),
        [], output_size = num_act, name = 'action_choice_mlp', reuse = (sub_idx > 0))
    action_prob_expanded = tf.reshape(action_prob_flat, [-1, max_num_actions + 1, num_act])
    action_probabilities_layer = tf.nn.softmax(action_prob_expanded, axis=-1)
    action_probabilities_presoftmax.append(action_prob_expanded)
    delta = mlp(
        subprogram_output, [512,], output_size = hidden_filters_encoder, name = 'global_transform',
        reuse = (sub_idx > 0)
    )
    last_encoding = last_encoding + delta
    encodings.append(last_encoding)
act_presoftmax = tf.stack(action_probabilities_presoftmax, 1)[:, :, :-1, :]
#batch, subprogram, timestep, action_selection
logprobabilities = tf.nn.log_softmax(act_presoftmax, -1)
act_presoftmax_flat = tf.reshape(act_presoftmax, [-1, 9, num_act])
mask_ph_flat = tf.reshape(mask_ph, [-1, max_actions_per_subprogram])
act_ind_flat = tf.reshape(act_ind, [-1, max_actions_per_subprogram])
ppl_loss = tf.contrib.seq2seq.sequence_loss(
    logits = act_presoftmax_flat,
    targets = act_ind_flat,
    weights = mask_ph_flat,
    average_across_timesteps=False,
    average_across_batch=False,
    softmax_loss_function=None,
    name=None
)
ppl_loss_avg = tf.reduce_mean(tf.pow(ppl_loss, 2.0)) * 10000 # + tf.reduce_mean(tf.pow(ppl_loss, 1.0)) * 100

tfvars = tf.trainable_variables()
weight_norm = tf.reduce_mean([tf.reduce_sum(tf.square(var)) for var in tfvars])*1e-3

action_taken = tf.argmax(act_presoftmax, -1, output_type=tf.int32)
correct_mat = tf.cast(tf.equal(action_taken, act_ind), tf.float32) * mask_ph
correct_percent = tf.reduce_sum(correct_mat, [1, 2])/tf.reduce_sum(mask_ph, [1, 2])
percent_correct = tf.reduce_mean(correct_percent)
percent_fully_correct = tf.reduce_mean(tf.cast(tf.equal(correct_percent, 1), tf.float32))

loss = ppl_loss_avg + weight_norm

opt_fcn = tf.train.AdamOptimizer(learning_rate=learning_rate)
#opt_fcn = tf.train.MomentumOptimizer(learning_rate=learning_rate, use_nesterov=True, momentum=.8)
optimizer, grad_norm_total = apply_clipped_optimizer(opt_fcn, loss)
sess = tf.Session()
sess.run(tf.global_variables_initializer())

0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [9]:
hidden_filters

256

In [10]:
encoding_last_layer

<tf.Tensor 'LeakyRelu_5/Maximum:0' shape=(?, 10, 512) dtype=float32>

In [ ]:
np.random.seed(0)
trn_percent = .1
num_samples = mask_np.shape[0]
ordered_samples = np.arange(num_samples)
np.random.shuffle(ordered_samples)
trn_samples = ordered_samples[:int(np.ceil(num_samples*trn_percent))]
val_samples_all = ordered_samples[int(np.ceil(num_samples*trn_percent)):]
val_samples = val_samples_all
trn_samples.shape, val_samples.shape

((2091,), (18819,))

In [ ]:
eval_itr = -1
bs = 32# trn_samples.shape[0]
for itr in range(1000000):
    if 1:#itr == 0:
        samples = np.random.choice(trn_samples, size = bs, replace = False)
        trn_feed_dict = {
            cmd_ind : cmd_np[samples],
            act_ind : act_np[samples],
            mask_ph : mask_np[samples],
            act_lengths : np.clip(struct_np[samples], a_min = 1, a_max = None),
            cmd_lengths : cmd_lengths_np[samples],
        }
        
    trn_feed_dict[learning_rate] = .02 / (np.power(itr + 10, .6))
    _, trn_loss, acc_trn_single, acc_trn = sess.run(
        [optimizer, loss, percent_correct, percent_fully_correct], trn_feed_dict)
    if itr == 0:
        trn_loss_avg = trn_loss
        acc_trn_avg = acc_trn
        acc_trn_single_avg = acc_trn_single
    else:
        trn_loss_avg = trn_loss_avg * .9 + trn_loss * .1
        acc_trn_avg = acc_trn_avg * .9 + acc_trn * .1
        acc_trn_single_avg = acc_trn_single_avg * .9 + acc_trn_single * .1
    if itr % 10 == 0 and itr > 0:
        # val_samples = np.random.choice(val_samples_all, size = bs, replace = False)
        eval_itr += 1
        val_feed_dict = {
            cmd_ind : cmd_np[val_samples],
            act_ind : act_np[val_samples],
            mask_ph : mask_np[val_samples],
            act_lengths : np.clip(struct_np[val_samples], a_min = 1, a_max = None),
            cmd_lengths : cmd_lengths_np[val_samples]
        }
        val_loss, acc_val = sess.run([loss, percent_fully_correct], val_feed_dict)
        if eval_itr == 0:
            val_loss_avg = val_loss
            acc_val_avg = acc_val
        else:
            val_loss_avg = val_loss_avg * .9 + val_loss * .1
            acc_val_avg = acc_val_avg * .9 + acc_val * .1
        print('itr:', itr, 'trn_loss', trn_loss_avg, 'val_loss', val_loss_avg)
        print('itr:', itr, 'trn_acc', acc_trn_avg, 
              'trn_single_acc', acc_trn_single_avg, 'val_acc', acc_val_avg)

itr: 10 trn_loss 15376.801653171487 val_loss 10507.9795
itr: 10 trn_acc 0.0 trn_single_acc 0.20511781186426153 val_acc 0.0
itr: 20 trn_loss 11901.973791987119 val_loss 10397.47490234375
itr: 20 trn_acc 0.0 trn_single_acc 0.2840895986319368 val_acc 0.0
itr: 30 trn_loss 9904.342040620428 val_loss 10152.272089843751
itr: 30 trn_acc 0.0 trn_single_acc 0.36601927021746095 val_acc 1.5941336459945888e-05
itr: 40 trn_loss 8644.19141127399 val_loss 9852.155037109376
itr: 40 trn_acc 0.0 trn_single_acc 0.43347304060092406 val_acc 3.0288539273897187e-05
itr: 50 trn_loss 6782.871077800957 val_loss 9817.064240429689
itr: 50 trn_acc 0.0 trn_single_acc 0.4917390515325421 val_acc 4.320102180645335e-05
itr: 60 trn_loss 6248.352906462718 val_loss 9389.95303123047
itr: 60 trn_acc 0.0 trn_single_acc 0.5045080123847749 val_acc 4.4194698081992105e-05
itr: 70 trn_loss 5359.990266903025 val_loss 8908.255042560548
itr: 70 trn_acc 0.0 trn_single_acc 0.5425655804719443 val_acc 0.0001832472520477313
itr: 80 trn_lo

itr: 530 trn_loss 748.6351244084332 val_loss 1172.220405916487
itr: 530 trn_acc 0.08379364212328509 trn_single_acc 0.8666399282640979 val_acc 0.04527738143362154
itr: 540 trn_loss 660.4174738902379 val_loss 1143.1794993580413
itr: 540 trn_acc 0.11274773620709473 trn_single_acc 0.88046474409635 val_acc 0.04740780770204261
itr: 550 trn_loss 714.4039650581824 val_loss 1117.7511734456748
itr: 550 trn_acc 0.0721681245354961 trn_single_acc 0.8736687587766542 val_acc 0.04795423683007176
itr: 560 trn_loss 685.8600486171317 val_loss 1085.8265932104823
itr: 560 trn_acc 0.08263102717235438 trn_single_acc 0.8711271970706883 val_acc 0.049737271247621916
itr: 570 trn_loss 647.4696222835325 val_loss 1061.4607734890435
itr: 570 trn_acc 0.08739534486769227 trn_single_acc 0.871791175223994 val_acc 0.051581121902152784
itr: 580 trn_loss 625.5207536587733 val_loss 1030.513573093264
itr: 580 trn_acc 0.10870949587984349 trn_single_acc 0.8802156547914136 val_acc 0.05373476885600527
itr: 590 trn_loss 613.3982

itr: 1040 trn_loss 387.4794924303482 val_loss 624.0942631034068
itr: 1040 trn_acc 0.18596437597774998 trn_single_acc 0.9128798585769347 val_acc 0.09703869300818661
itr: 1050 trn_loss 373.8172701271253 val_loss 626.6522501231442
itr: 1050 trn_acc 0.17255263574571683 trn_single_acc 0.9148466753486532 val_acc 0.0963044823205729
itr: 1060 trn_loss 396.4156826866688 val_loss 622.9939281870016
itr: 1060 trn_acc 0.1994747958732101 trn_single_acc 0.9108914419617593 val_acc 0.09843342626061377
itr: 1070 trn_loss 380.8290513781011 val_loss 621.1832499679108
itr: 1070 trn_acc 0.1814418126674619 trn_single_acc 0.9064361474938972 val_acc 0.10005190408795778
itr: 1080 trn_loss 366.02386047772154 val_loss 620.6642841019791
itr: 1080 trn_acc 0.20906261322543207 trn_single_acc 0.909831470255533 val_acc 0.10072209480318163
itr: 1090 trn_loss 372.0161121039871 val_loss 619.726926736703
itr: 1090 trn_acc 0.16094698102829832 trn_single_acc 0.908825706576807 val_acc 0.10243053251128853
itr: 1100 trn_loss 39

itr: 1550 trn_loss 293.4449763098491 val_loss 570.213191812684
itr: 1550 trn_acc 0.2956086179194739 trn_single_acc 0.928117846854168 val_acc 0.13317075089449948
itr: 1560 trn_loss 308.10695975101083 val_loss 571.5689539302438
itr: 1560 trn_acc 0.2976027984544041 trn_single_acc 0.9242682061084069 val_acc 0.1331168670877987
itr: 1570 trn_loss 365.7582955939165 val_loss 569.2391398360476
itr: 1570 trn_acc 0.27927257888135143 trn_single_acc 0.9223306703578393 val_acc 0.13248916948662764
itr: 1580 trn_loss 329.3290584723067 val_loss 577.2561255106459
itr: 1580 trn_acc 0.2681541136076789 trn_single_acc 0.9251009462242415 val_acc 0.1332951976492396
itr: 1590 trn_loss 306.06053999206375 val_loss 581.621144063097
itr: 1590 trn_acc 0.27160562845474867 trn_single_acc 0.9269081888477925 val_acc 0.133382969525551
itr: 1600 trn_loss 322.82267654450163 val_loss 582.3336207212404
itr: 1600 trn_acc 0.29688120026448206 trn_single_acc 0.923202303116589 val_acc 0.13351510175504852
itr: 1610 trn_loss 321.0

itr: 2060 trn_loss 240.92972255432494 val_loss 628.9641038693019
itr: 2060 trn_acc 0.39907387877766887 trn_single_acc 0.936635313528392 val_acc 0.17195186850678854
itr: 2070 trn_loss 240.36092136478533 val_loss 627.8300286874498
itr: 2070 trn_acc 0.40269123929935413 trn_single_acc 0.9411155459100655 val_acc 0.1741307184450501
itr: 2080 trn_loss 233.71372413213646 val_loss 627.977494568705
itr: 2080 trn_acc 0.44135795121395965 trn_single_acc 0.9430001196042366 val_acc 0.1748376314657337
itr: 2090 trn_loss 256.4408348560603 val_loss 629.8318447212094
itr: 2090 trn_acc 0.3798557591050154 trn_single_acc 0.939305825810574 val_acc 0.17576611114895996
itr: 2100 trn_loss 244.11446174141182 val_loss 628.3270782178386
itr: 2100 trn_acc 0.40080075428836315 trn_single_acc 0.9403526846266355 val_acc 0.17744663423320642
itr: 2110 trn_loss 226.26941442741042 val_loss 632.4578530620703
itr: 2110 trn_acc 0.4344885900461699 trn_single_acc 0.946408883100166 val_acc 0.17898567377943686
itr: 2120 trn_loss 

...

itr: 32210 trn_loss 0.07152177710657036 val_loss 76.97529555892251
itr: 32210 trn_acc 0.9999998543009259 trn_single_acc 0.9999999890306636 val_acc 0.9786088097756613
itr: 32220 trn_loss 0.743018463567018 val_loss 73.59281231345507
itr: 32220 trn_acc 0.998654739166624 trn_single_acc 0.9998777057537249 val_acc 0.9795682694314876
itr: 32230 trn_loss 0.30391824250294713 val_loss 70.77561444087421
itr: 32230 trn_acc 0.9995309365510908 trn_single_acc 0.9999573586329756 val_acc 0.9803999005972408
itr: 32240 trn_loss 0.14968240603071797 val_loss 67.45895234981413
itr: 32240 trn_acc 0.9998364476883265 trn_single_acc 0.9999851318746623 val_acc 0.9812493329542038
itr: 32250 trn_loss 0.09764207480325537 val_loss 64.67381074642451
itr: 32250 trn_acc 0.9999429728350909 trn_single_acc 0.99999481580525 val_acc 0.9820244495836339
itr: 32260 trn_loss 0.07757088952988951 val_loss 64.1713958735643
itr: 32260 trn_acc 0.9999801158570962 trn_single_acc 0.9999981923830613 val_acc 0.9825732694358327
itr: 32270

itr: 32710 trn_loss 0.3054684993858819 val_loss 33.48668553051816
itr: 32710 trn_acc 0.9993565902122041 trn_single_acc 0.9999597868882626 val_acc 0.9896176153900778
itr: 32720 trn_loss 0.14907894086227905 val_loss 33.94603928581595
itr: 32720 trn_acc 0.9997756568788463 trn_single_acc 0.9999859785549277 val_acc 0.9894177461697956
itr: 32730 trn_loss 0.09446414707807571 val_loss 36.33048694720017
itr: 32730 trn_acc 0.9999217763904689 trn_single_acc 0.9999951110244042 val_acc 0.9891794095964105
itr: 32740 trn_loss 0.07537037741431574 val_loss 37.026478779823904
itr: 32740 trn_acc 0.9999727251138497 trn_single_acc 0.9999982953196156 val_acc 0.9891508910534565
itr: 32750 trn_loss 0.06869804660106157 val_loss 36.73956004307687
itr: 32750 trn_acc 0.9999904898352432 trn_single_acc 0.9999994056147026 val_acc 0.9892049276957916
itr: 32760 trn_loss 0.06635703412149474 val_loss 36.04779264503139
itr: 32760 trn_acc 0.9999966840105875 trn_single_acc 0.9999997927506616 val_acc 0.9893385807392008
itr:

itr: 33210 trn_loss 0.06387665366940587 val_loss 25.45819154228418
itr: 33210 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.991392932486716
itr: 33220 trn_loss 0.06385300921786925 val_loss 25.511924542596777
itr: 33220 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9913821776081769
itr: 33230 trn_loss 0.0638012080253101 val_loss 25.58872738863007
itr: 33230 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9913724982174917
itr: 33240 trn_loss 0.06377585206418804 val_loss 25.663285321458957
itr: 33240 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.991347848483862
itr: 33250 trn_loss 0.0637289377436239 val_loss 25.73574177008699
itr: 33250 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9913203469892814
itr: 33260 trn_loss 0.06371240627694554 val_loss 25.805947538451825
itr: 33260 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9912955956441588
itr: 

itr: 33710 trn_loss 0.062290704845420236 val_loss 28.406328162129444
itr: 33710 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9903707979810292
itr: 33720 trn_loss 0.06223195548933435 val_loss 28.399575007476557
itr: 33720 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9903453539632607
itr: 33730 trn_loss 0.062216688906751076 val_loss 28.427671324477927
itr: 33730 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9903330818554327
itr: 33740 trn_loss 0.06216053170089867 val_loss 28.47779722028941
itr: 33740 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9903167261845379
itr: 33750 trn_loss 0.062114770816893955 val_loss 28.527061870819065
itr: 33750 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.990312633588896
itr: 33760 trn_loss 0.06207744687797999 val_loss 28.61969088118589
itr: 33760 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.990293006010340

itr: 34210 trn_loss 0.0702338990367614 val_loss 90.50931463736467
itr: 34210 trn_acc 0.9999985332258956 trn_single_acc 0.9999999017833721 val_acc 0.981346923600374
itr: 34220 trn_loss 0.06655895474940213 val_loss 93.74080733744657
itr: 34220 trn_acc 0.9999994885674932 trn_single_acc 0.9999999657539794 val_acc 0.9811239169455216
itr: 34230 trn_loss 0.06478147176652216 val_loss 90.24843360443434
itr: 34230 trn_acc 0.9999998216745113 trn_single_acc 0.999999988059151 val_acc 0.9809869760051352
itr: 34240 trn_loss 0.06350298026029322 val_loss 89.73504959457685
itr: 34240 trn_acc 0.9999999378217469 trn_single_acc 0.9999999958364835 val_acc 0.9808690399326369
itr: 34250 trn_loss 0.0656678196507361 val_loss 83.7111107133052
itr: 34250 trn_acc 0.9999999783197837 trn_single_acc 0.9999999985482715 val_acc 0.9816981241462366
itr: 34260 trn_loss 0.06353351369469107 val_loss 79.0301508565743
itr: 34260 trn_acc 0.9999999924405759 trn_single_acc 0.9999999994938136 val_acc 0.9823752241156462
itr: 34270

In [ ]:
correct_percent.shape, percent_fully_correct.shape

In [ ]:
10, None, 7, 9, None

In [ ]:
cmd_np.shape, act_np.shape, mask_np.shape, struct_np.shape, cmd_lengths_np.shape

In [ ]:
cmd_ind.shape, act_ind.shape, mask_ph.shape, act_lengths.shape, cmd_lengths.shape

In [ ]:
trn_samples.shape, val_samples.shape

In [ ]:

act_presoftmax = tf.stack(action_probabilities_presoftmax, 1)[:, :, :-1, :]
#batch, subprogram, timestep, action_selection
logprobabilities = tf.nn.log_softmax(act_presoftmax, -1)
act_presoftmax_flat = tf.reshape(act_presoftmax, [-1, 9, num_act])
mask_ph_flat = tf.reshape(mask_ph, [-1, max_actions_per_subprogram])
act_ind_flat = tf.reshape(act_ind, [-1, max_actions_per_subprogram])

In [ ]:
act_presoftmax_flat = tf.reshape(act_presoftmax, [-1, 9, num_act])

In [ ]:
max_actions_per_subprogram

In [ ]:
sess.run(act_presoftmax, feed_dict).shape

In [ ]:
action_map

In [ ]:
print(*actions_ind[2])

In [ ]:
command_map

In [ ]:
action_map

In [ ]:
subprogram_output

In [ ]:
subprogram_last_layer[:,cmd_lengths,:]

In [ ]:
encoding_last_layer

In [ ]:
tf.gather(
    encoding_last_layer,
    [1,2],
    axis=1
)

In [ ]:
tf.gather_nd(
    encoding_last_layer,
    np.array([[0,1,2,3,4], [1,4,3,2,5]]).T,
    name=None
)

In [ ]:
cmd_lengths

In [ ]:
def generate_command(sub_cmd, num_repeat):
    return sub_cmd * num_repeat

In [ ]:
def process_command(cmd):
    

In [ ]:
uni_commands

In [ ]:
uni_actions

In [ ]:
uni_tokens

In [ ]:
df.shape